In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import os, warnings 
warnings.filterwarnings('ignore') 
import stan

In [2]:
path = Path('outputs_stan')
if not os.path.exists(path):
    os.makedirs(path)

In [3]:
def stan_wrap(adata, lam_range=[1e-3, 1e3], n_steps=4, use_pixel=True):
    adata = stan.add_gene_tf_matrix(adata, min_cells_proportion = 0.2, min_tfs_per_gene= 5, min_genes_per_tf= 10,
                                    gene_tf_source="hTFtarget", tf_list="humantfs", source_dir="resources/")
    stan.pixel_intensity(adata, windowsize=25)
    if use_pixel:
        stan.make_kernel_from_pixel(adata, n=250, im_feats_weight=0.1)
    else:
        stan.make_kernel(adata, X=adata.obsm['spatial'], n=250)
    sc.pp.normalize_total(adata)
    adata.layers['scaled'] = np.sqrt(adata.to_df())
    stan.assign_folds(adata, n_folds=10, random_seed=0)
    
    stan_model = stan.Stan(adata, layer='scaled')
    stan_model.fit(n_steps=n_steps, stages=1,
                   grid_search_params={'lam1':lam_range, 'lam2':lam_range})
    print(stan_model.params)
    
    cor, gene_cor = stan_model.evaluate(fold=-1)
    adata.obs['pred_cor_stan'] = cor
    adata.var['pred_cor_stan'] = gene_cor
    print("Spot-wise correlation:" + str(round(np.nanmedian(cor), 4)))
    print("Gene-wise correlation: " + str(round(np.nanmedian(gene_cor), 4)))
    
    adata.obsm['tfa_stan'] = pd.DataFrame(
        stan_model.W_concat.T, index=adata.obs_names, columns=adata.uns['tf_names'])
    return adata

### CytAssist (Visium)

In [4]:
sample_list = ["CytAssist_FFPE_Protein_Expression_Human_Tonsil",
               "CytAssist_FFPE_Protein_Expression_Human_Tonsil_AddOns",
              'CytAssist_FFPE_Protein_Expression_Human_Breast_Cancer']
for sample_id in sample_list:
    print(sample_id)
    adata, pdata = stan.read_cytassist('data/'+sample_id, min_cells=5, min_counts=500, remove_isotype=True)
    adata = stan_wrap(adata)
    adata.write(path / ('adata_'+sample_id+'.h5ad'))

CytAssist_FFPE_Protein_Expression_Human_Tonsil
Time elapsed: 31.76 seconds
{'lam1': 1000.0, 'lam2': 0.001}
Spot-wise correlation:0.3161
Gene-wise correlation: 0.1154
CytAssist_FFPE_Protein_Expression_Human_Tonsil_AddOns
Time elapsed: 37.32 seconds
{'lam1': 1000.0, 'lam2': 1000.0}
Spot-wise correlation:0.34
Gene-wise correlation: 0.1697
CytAssist_FFPE_Protein_Expression_Human_Breast_Cancer
Time elapsed: 27.82 seconds
{'lam1': 1000.0, 'lam2': 0.001}
Spot-wise correlation:0.1988
Gene-wise correlation: 0.1716


### Breast

In [5]:
sample_list = ["1142243F", "1160920F", "CID4290", "CID4535", "CID4465", "CID44971"]
for sample_id in sample_list:
    print(sample_id)
    adata = stan.read_breast_wu("data/Breast_Wu/{}.h5ad".format(sample_id))
    adata = stan_wrap(adata)
    adata.write(path / ('adata_Breast_Wu_'+sample_id+'.h5ad'))

1142243F
Time elapsed: 24.37 seconds
{'lam1': 1000.0, 'lam2': 1000.0}
Spot-wise correlation:0.2199
Gene-wise correlation: 0.116
1160920F
Time elapsed: 27.31 seconds
{'lam1': 1000.0, 'lam2': 10.0}
Spot-wise correlation:0.223
Gene-wise correlation: 0.1575
CID4290
Time elapsed: 12.51 seconds
{'lam1': 1000.0, 'lam2': 10.0}
Spot-wise correlation:0.2531
Gene-wise correlation: 0.1728
CID4535
Time elapsed: 7.14 seconds
{'lam1': 1000.0, 'lam2': 10.0}
Spot-wise correlation:0.2937
Gene-wise correlation: 0.1514
CID4465
Time elapsed: 7.99 seconds
{'lam1': 1000.0, 'lam2': 1000.0}
Spot-wise correlation:0.2378
Gene-wise correlation: 0.1563
CID44971
Time elapsed: 7.19 seconds
{'lam1': 1000.0, 'lam2': 0.001}
Spot-wise correlation:0.2834
Gene-wise correlation: 0.1897


### Lymphnode

In [6]:
sample_id = "V1_Human_Lymph_Node"
adata = stan.read_visium_sge(sample_id=sample_id, min_cells=5, min_counts=5000)
adata = stan_wrap(adata)
adata.write(path / ('adata_'+sample_id+'.h5ad'))

Time elapsed: 27.52 seconds
{'lam1': 1000.0, 'lam2': 1000.0}
Spot-wise correlation:0.2277
Gene-wise correlation: 0.0896


### Glioblastoma

In [7]:
sample_id = "Parent_Visium_Human_Glioblastoma"
adata = stan.read_visium_sge(sample_id=sample_id, min_cells=5, min_counts=500)
adata = stan_wrap(adata)
adata.write(path / ('adata_'+sample_id+'.h5ad'))

Time elapsed: 19.26 seconds
{'lam1': 1000.0, 'lam2': 0.001}
Spot-wise correlation:0.2424
Gene-wise correlation: 0.1636
